In [1]:
import pandas as pd
import dask.dataframe as dd
import random
from distributed import Client

In [2]:
c = Client()

In [3]:
all_rows = []
for path in range(10):
    for date in ['2021-07-01', '2021-10-01', '2022-01-01']:
        all_rows.append([date, path, random.random()*200-100]) 

In [4]:
df = pd.DataFrame(all_rows, columns=['date', 'path', 'mtm']).astype({'path': 'int32', 'mtm': 'float64'}).set_index('path')
df.date = pd.to_datetime(df.date)
display(df.dtypes)
df.head()

date    datetime64[ns]
mtm            float64
dtype: object

,date,mtm
path,,
0,2021-07-01,22.068791
0,2021-10-01,92.160839
0,2022-01-01,58.258004
1,2021-07-01,-43.255481
1,2021-10-01,-9.196393


In [5]:
ddf = dd.from_pandas(df, npartitions=3)
ddf

,date,mtm
npartitions=3,,
0,datetime64[ns],float64
4,...,...
7,...,...
9,...,...


In [6]:
trade_filepath = 'row_partitioned_trade_file.parquet'
ddf.to_parquet(trade_filepath)
del ddf

In [15]:
ddf = dd.read_parquet(trade_filepath)

In [13]:
a = ddf.persist()

In [14]:
del a

In [15]:
%%time
df1 = ddf.loc[0:3, :].compute()
display(f'{df1.memory_usage(deep=True).sum()/2**10} kb')
print('Measured Times:')

'0.28125 kb'

Measured Times:
CPU times: user 21.5 ms, sys: 2.04 ms, total: 23.5 ms
Wall time: 30.6 ms


In [16]:
%%time
df2 = ddf.loc[0:6, :].compute()
display(f'{df2.memory_usage(deep=True).sum()/2**10} kb')
print('Measured Times:')

'0.4921875 kb'

Measured Times:
CPU times: user 39 ms, sys: 4.39 ms, total: 43.4 ms
Wall time: 51.4 ms


In [17]:
%%time
df3 = ddf.loc[0:, :].compute()
display(f'{df3.memory_usage(deep=True).sum()/2**10} kb')
print('Measured Times:')

'0.703125 kb'

Measured Times:
CPU times: user 20.3 ms, sys: 2.04 ms, total: 22.3 ms
Wall time: 27.4 ms
